In [108]:
import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.io as pio
import numpy as np
import pandas as pd

population_202104 = 23514196
csv_filename = '~/Programs/VaccineCOVID19Taiwan/VaccineCOVID19Taiwan - public.csv'
Date = '日期'
Event = '事件'
InjectedAmount = '累計施打'
InjectedAmountCorrect = '校正累計施打'
ConfirmedLocalDaily = '單日本土確診'
DeathDaily = '單日死亡'
FinalCorrectAmount = '完成本土增補'


first_date = datetime.datetime.fromisoformat('2021-03-01')
last_date = datetime.datetime.today() + datetime.timedelta(days=1)
dpi = 200


In [85]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def verticalizeText(text):
    text = text.strip()
    text_v = ''        
    isE = [isEnglish(t) for t in text]
    for j in range(len(text)):
        if text[j] == ' ':
            continue
        if j < len(text)-1:
            if text[j] == '/':
                enter = True
            elif j < len(text)-2:
                if text[j+1] == '/':
                    enter = True
                elif text[j+1] == '#':
                    enter = True
                elif isE[j] and isE[j+1]:
                    enter = False
                elif isE[j] and not isE[j+1]:
                    enter = True
                elif not isE[j] and isE[j+1]:
                    enter = True
                elif not isE[j] and not isE[j+1]:
                    enter = True
            elif isE[j] and isE[j+1]:
                enter = False
            elif isE[j] and not isE[j+1]:
                enter = True
            elif not isE[j] and isE[j+1]:
                enter = True
            elif not isE[j] and not isE[j+1]:
                enter = True
        else:
            enter = False

        if enter:
            text_v += text[j] + '\n'
        else: 
            text_v += text[j]
    
    return text_v

In [86]:

df = pd.read_csv(csv_filename)
df = df[~df[Date].isnull()]
df.loc[:,Date] = df[Date].astype('datetime64[ns]')
df.set_index(Date,inplace=True)
ind = (df.index >= first_date + datetime.timedelta(days=-1)) * (df.index <= last_date.strftime('%Y-%m-%d'))
df = df[ind]
df.loc[(last_date+datetime.timedelta(days=-1)).strftime('%Y-%m-%d'),InjectedAmountCorrect] = \
    df.loc[(last_date+datetime.timedelta(days=-1)).strftime('%Y-%m-%d'),InjectedAmount]

ind = df[InjectedAmountCorrect].isnull()
df.loc[ind,InjectedAmountCorrect] = df.loc[ind,InjectedAmount]
df['mavg'] = df[InjectedAmountCorrect].interpolate()


In [135]:
vaccine_color = '#6F00D2'
event_color = '#000000'
# Create traces
#fig = go.Figure()
#fig = make_subplots(rows=1, cols=1, shared_xaxes=True,specs=[[{"secondary_y": True}]])
fig = make_subplots(shared_xaxes=True,specs=[[{"secondary_y": True}]])

hovertemplate = '%{y:.2f}%<extra></extra>'
ind = (~df[InjectedAmountCorrect].isnull()) & (df.index < (last_date + datetime.timedelta(days=-1)))
x = df.index[ind]
y = df[InjectedAmountCorrect][ind]/population_202104*100
fig.add_trace(
    go.Scatter(
        x=x, 
        y=y,
        name=InjectedAmountCorrect,
        mode='lines',
        line=dict(color=vaccine_color, width=2, dash='dot'),
        hovertemplate=hovertemplate
    ), secondary_y=True)
fig.update_layout(hovermode="x")
#fig.update_layout(hovermode="x unified")

#hovertemplate = '%{x}<br>%{text}<extra></extra>'
hovertemplate = '%{text}<extra></extra>'
ind = ~df[Event].isnull() & (df.index < (last_date + datetime.timedelta(days=-1)))
x = df.index[ind]
y = df.loc[ind,"mavg"]/population_202104*100
label = df.loc[ind,Event].to_numpy()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        name=Event,
        mode='markers',
        text=label,
        marker={"size": 5, "color":event_color},
        hovertemplate=hovertemplate,
    ), secondary_y=True)

#hovertemplate = 'Date=%{x}<br>Diff=%{y}<extra></extra>'
hovertemplate = '%{text}人<extra></extra>'

ind = ~ (df[DeathDaily] == 0)
death_color = 'red'
fig.add_trace(
    go.Bar(
        x=df.index[ind],y=df[DeathDaily][ind],
        name=DeathDaily,
        text=df[DeathDaily][ind],
        marker_color=death_color,            
        hovertemplate=hovertemplate,
    ), secondary_y=False)

ind = ~ (df[ConfirmedLocalDaily] == 0)
confirmd_color = '#FFAD86'
fig.add_trace(
    go.Bar(
        x=df.index[ind],y=(df[ConfirmedLocalDaily]-df[DeathDaily])[ind],
        name=ConfirmedLocalDaily,
        text=df[ConfirmedLocalDaily][ind],
        marker_color=confirmd_color,
        hovertemplate=hovertemplate,
    ), secondary_y=False)

ind = ~ (df[FinalCorrectAmount] == 0)
correction_color = 'gold'
fig.add_trace(
    go.Bar(
        x=df.index[ind],y=df[FinalCorrectAmount][ind],
        name=FinalCorrectAmount[2:],
        text=df[FinalCorrectAmount][ind],
        marker_color=correction_color,
        hovertemplate=hovertemplate,
    ), secondary_y=False)

fig.update_layout(barmode='stack')

#fig.add_trace(go.Scatter(x=random_x, y=random_y0,
#                    mode='lines',
#                    name='lines'))
#fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                    mode='lines+markers',
#                    name='lines+markers'))
#fig.add_trace(go.Scatter(x=random_x, y=random_y2,
#                    mode='markers', name='markers'))

fig.update_layout(title_text="Double Y Axis Example",width=1200, height=675)
fig.show()
fig.update_layout(width=1600, height=900)
pio.write_html(fig, file='_includes/figure.html', auto_open=False)